# 수집된 제품 link로 제품 세부 정보를 수집

In [1]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

In [2]:
v4_path = os.getcwd() + '/' + 'glow_pick_product_info_v4.3.csv'
v4 = pd.read_csv(v4_path)

In [ ]:
df_brd_w_prd_v2 = pd.read_csv("df_brd_w_prd_v2.csv")
df_brd_w_prd_v2  = df_brd_w_prd_v2.iloc[:, 1:]

# **제품 정보 수집**

In [12]:
def scrap_product_details(prd_link):
    
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    print(userAgent)

    #options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    options.add_argument(f'user-agent={userAgent}')
    wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

    wd.get(prd_link)
    time.sleep(5)
    wd.maximize_window()
    
    socket.setdefaulttimeout(30)
        
    try:
        
        wd.find_element_by_xpath('//*[@id=\"default-layout\"]/div/div[1]/span/div/div[2]/div[2]/button[2]').click()
        time.sleep(5)

        html = wd.page_source
        bs = BeautifulSoup(html, 'html.parser')
        
        time.sleep(3)
        img_src = wd.find_element_by_xpath("/html/body/div/div/div/div/main/div/section/div[1]/div/div/img").get_attribute('src')
        img_src = re.sub("w=[0-9]*", "w=500", img_src)
        
        time.sleep(3)
        brd_name = wd.find_element_by_class_name("product__summary__brand__name").text
        brd_id = np.nan
        brd_link = np.nan

        prd_name = wd.find_element_by_class_name("product__summary__name").text
        prd_id = prd_link.split('/')[-1]

        prd_volume = wd.find_element_by_class_name("offer__volume-price").text.split(' / ')[0]
        prd_price = wd.find_element_by_class_name("offer__volume-price").text.split(' / ')[1]

        time.sleep(2)

        element = wd.find_element_by_xpath('/html/body/div/div/div/div/main/div/section/div[3]')
        wd.execute_script("arguments[0].scrollIntoView(true);", element)

        time.sleep(2)
        info_article = bs.select(".info__article")


        #랭킹
        if '랭킹' in str(bs.select(".info__article")):
            prd_grp_text = wd.find_element_by_class_name("rank-item__name").text #유사제품군랭킹(text)
            prd_grp_rank = wd.find_elements_by_class_name("rank-item__rank")[0].text #유사제품군랭킹(text)
            prd_brd_rank = wd.find_elements_by_class_name("rank-item__rank")[1].text #브랜드 내 링킹(int)

            if '수상' in str(bs.select(".info__article")):
                view_all_award_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[2]/h3/button'
                view_all_ingredient_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[3]/h3/button'
                view_all_prd_desc_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[4]/h3/button'
            else:
                view_all_award_xpath = ''
                view_all_ingredient_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[2]/h3/button'
                view_all_prd_desc_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[3]/h3/button'


        else: #랭킹이 없는 경우

            prd_grp_text = ''
            prd_grp_rank = ''
            prd_brd_rank = ''

            if '수상' in str(bs.select(".info__article")):
                view_all_award_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[1]/h3/button'
                view_all_ingredient_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[2]/h3/button'
                view_all_prd_desc_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[3]/h3/button'

            else:
                view_all_award_xpath = ''
                view_all_ingredient_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[1]/h3/button'
                view_all_prd_desc_xpath = '/html/body/div/div/div/div/main/div/section/div[3]/article[2]/h3/button'


        #수상내역 전체보기
        award_name = [] #제품 수상명
        award_part = [] #제품 수상 부문
        award_rank = [] #제품 수상 순위

        if view_all_award_xpath != '':
            wd.find_element_by_xpath(view_all_award_xpath).click()
            html = wd.page_source
            bs = BeautifulSoup(html, 'html.parser')
            time.sleep(5)

            for i in range(len(bs.select('.awards__item__text__name'))):
                award_name.append(bs.select('.awards__item__text__name')[i].text.strip())
                award_part.append(bs.select('.awards__item__text__award')[i].text.strip())
                award_rank.append(bs.select(".awards__item__text__rank")[i].text.strip())
        #     award_name = re.findall('<p class="awards__item__text__name" (.*?)id="(.*?)">', bs.select('.awards__item__text__name')) #제품 수상명
        #     #re.findall('<div class="products__product product" (.*?)id="(.*?)">', str(product_lst[0])
        #     award_part = bs.select('.awards__item__text__award') #제품 수상 부문
        #     award_rank = bs.select(".awards__item__text__rank") #제품 수상 순위
            wd.find_element_by_xpath('/html/body/div/div/div/div/div[1]/span/div/div[2]/h1/button/span').click()
        time.sleep(2)


        #성분 구성 전체보기
        wd.find_element_by_xpath(view_all_ingredient_xpath).click()
        html = wd.page_source
        bs = BeautifulSoup(html, 'html.parser')
        time.sleep(5)

        ingredient_unknown_cnt = wd.find_elements_by_class_name("tag__bg__label")[0].text #위험도 미정 개수
        ingredient_low_cnt = wd.find_elements_by_class_name("tag__bg__label")[1].text #낮은 위험도 개수
        ingredient_mid_cnt = wd.find_elements_by_class_name("tag__bg__label")[2].text #중간 위험도 개수
        ingredient_high_cnt = wd.find_elements_by_class_name("tag__bg__label")[3].text #높은 위험도 개수

        html = wd.page_source
        bs = BeautifulSoup(html, 'html.parser')
        time.sleep(3)

        ingredient_lst_unknown = []
        ingredient_lst_low = []
        ingredient_lst_mid = []
        ingredient_lst_high = []

        ingredient_lst = bs.select('.ingredient__list__item.item')

        for index,li in enumerate(ingredient_lst):
            if li.find("div",{"class" : "tag__bg tag__bg--item tag__bg--unknown"}):
                ingredient_lst_unknown.append(li)
            if li.find("div",{"class" : "tag__bg tag__bg--item tag__bg--low"}):
                ingredient_lst_low.append(li)
            if li.find("div",{"class" : "tag__bg tag__bg--item tag__bg--mid"}):
                ingredient_lst_mid.append(li)
            if li.find("div",{"class" : "tag__bg tag__bg--item tag__bg--high"}):
                ingredient_lst_high.append(li)

        ingredient_unknown_kor_lst = []
        ingredient_unknown_eng_lst = []
        ingredient_unknown_desc_lst = []

        for ingredient_unknown in ingredient_lst_unknown:

            if ingredient_unknown.find("p",{"class" : "item__wrapper__text__kor"}):
                ingredient_unknown_kor = ingredient_unknown.find("p",{"class" : "item__wrapper__text__kor"}).text
                ingredient_unknown_kor = ingredient_unknown_kor.strip()
                ingredient_unknown_kor_lst.append(ingredient_unknown_kor)

            if ingredient_unknown.find("p",{"class" : "item__wrapper__text__eng"}):
                ingredient_unknown_eng = ingredient_unknown.find("p",{"class" : "item__wrapper__text__eng"}).text
                ingredient_unknown_eng = ingredient_unknown_eng.strip()
                ingredient_unknown_eng_lst.append(ingredient_unknown_eng)


            if ingredient_unknown.find("p",{"class" : "item__wrapper__text__desc"}):
                ingredient_unknown_desc = ingredient_unknown.find("p",{"class" : "item__wrapper__text__desc"}).text
                ingredient_unknown_desc = ingredient_unknown_desc.strip()
                ingredient_unknown_desc_lst.append(ingredient_unknown_desc)

        ingredient_low_kor_lst = []
        ingredient_low_eng_lst = []
        ingredient_low_desc_lst = []

        for ingredient_low in ingredient_lst_low:

            if ingredient_low.find("p",{"class" : "item__wrapper__text__kor"}):
                ingredient_low_kor = ingredient_low.find("p",{"class" : "item__wrapper__text__kor"}).text
                ingredient_low_kor = ingredient_low_kor.strip()
                ingredient_low_kor_lst.append(ingredient_low_kor)

            if ingredient_low.find("p",{"class" : "item__wrapper__text__eng"}):
                ingredient_low_eng = ingredient_low.find("p",{"class" : "item__wrapper__text__eng"}).text
                ingredient_low_eng = ingredient_low_eng.strip()
                ingredient_low_eng_lst.append(ingredient_low_eng)


            if ingredient_low.find("p",{"class" : "item__wrapper__text__desc"}):
                ingredient_low_desc = ingredient_low.find("p",{"class" : "item__wrapper__text__desc"}).text
                ingredient_low_desc = ingredient_low_desc.strip()
                ingredient_low_desc_lst.append(ingredient_low_desc)

        ingredient_mid_kor_lst = []
        ingredient_mid_eng_lst = []
        ingredient_mid_desc_lst = []

        for ingredient_mid in ingredient_lst_mid:

            if ingredient_mid.find("p",{"class" : "item__wrapper__text__kor"}):
                ingredient_mid_kor = ingredient_mid.find("p",{"class" : "item__wrapper__text__kor"}).text
                ingredient_mid_kor = ingredient_mid_kor.strip()
                ingredient_mid_kor_lst.append(ingredient_mid_kor)

            if ingredient_mid.find("p",{"class" : "item__wrapper__text__eng"}):
                ingredient_mid_eng = ingredient_mid.find("p",{"class" : "item__wrapper__text__eng"}).text
                ingredient_mid_eng = ingredient_mid_eng.strip()
                ingredient_mid_eng_lst.append(ingredient_mid_eng)


            if ingredient_mid.find("p",{"class" : "item__wrapper__text__desc"}):
                ingredient_mid_desc = ingredient_mid.find("p",{"class" : "item__wrapper__text__desc"}).text
                ingredient_mid_desc = ingredient_mid_desc.strip()
                ingredient_mid_desc_lst.append(ingredient_mid_desc)


        ingredient_high_kor_lst = []
        ingredient_high_eng_lst = []
        ingredient_high_desc_lst = []

        for ingredient_high in ingredient_lst_high:

            if ingredient_high.find("p",{"class" : "item__wrapper__text__kor"}):
                ingredient_high_kor = ingredient_high.find("p",{"class" : "item__wrapper__text__kor"}).text
                ingredient_high_kor = ingredient_high_kor.strip()
                ingredient_high_kor_lst.append(ingredient_high_kor)

            if ingredient_high.find("p",{"class" : "item__wrapper__text__eng"}):
                ingredient_high_eng = ingredient_high.find("p",{"class" : "item__wrapper__text__eng"}).text
                ingredient_high_eng = ingredient_high_eng.strip()
                ingredient_high_eng_lst.append(ingredient_high_eng)


            if ingredient_high.find("p",{"class" : "item__wrapper__text__desc"}):
                ingredient_high_desc = ingredient_high.find("p",{"class" : "item__wrapper__text__desc"}).text
                ingredient_high_desc = ingredient_high_desc.strip()
                ingredient_high_desc_lst.append(ingredient_high_desc)

        wd.find_element_by_xpath('/html/body/div/div/div/div/div[1]/span/div/div[2]/h1/button/span').click()
        time.sleep(2)


        #제품 설명 전체보기
        wd.find_element_by_xpath(view_all_prd_desc_xpath).click()

        html = wd.page_source
        bs = BeautifulSoup(html, 'html.parser')
        time.sleep(5)

        ##제품 설명
        try:
            prd_desc = wd.find_element_by_class_name('descriptions__article__pre').text.replace("\n", '')
            time.sleep(2)
        except:
            prd_desc = np.nan

        ##카테고리
        category = []
        try:
            category.append(wd.find_element_by_class_name("descriptions__article__category").text)
        except:
            pass
        
        try:
            category_link = bs.select('.descriptions__article__category.descriptions__article__category-link')#제품 카테고리(하위)
            category_link = re.findall(">(.*?)</span>", str(category_link))
        except:
            category_link = []
            
        category = category + category_link

        ##컬러/타입
        try:
            color_type = wd.find_element_by_xpath('/html/body/div/div/div/div/div[1]/span/div/div[2]/div/article[2]/div').text
            time.sleep(3)
        except:
            color_type=''
            pass

        ##판매처
        try:
            store = str(bs.select('.stores__store__name')).replace(' ', '')
            store = re.findall("\n(.*?)\n", store)
            time.sleep(3)
        except:
            store = []

        wd.find_element_by_xpath('/html/body/div/div/div/div/div[1]/span/div/div[2]/h1/button/span').click()
        time.sleep(2)

        #리뷰
        element = wd.find_element_by_xpath('/html/body/div/div/div/div/main/div/section/section')
        wd.execute_script("arguments[0].scrollIntoView(true);", element)
        time.sleep(5)

        review = []
        rating = []
        review_date = []
        user_review_cnt = []
        user_skin_type = []
        user_age = []
        user_sex = []
        user_name = []

        review_len = len(wd.find_elements_by_class_name("cutter__pre"))
        info_details = bs.select(".info__details")
        time.sleep(5)

        for i in range(review_len):
            review.append(wd.find_elements_by_class_name("cutter__pre")[i].text.replace('\n', ''))
            review_date.append(bs.select(".review__side-info__created-at")[i].text.strip())
            user_age.append(str(info_details[i]).split('\n')[1].strip())
            user_skin_type.append(str(info_details[i]).split('\n')[3].strip())
            user_sex.append(str(info_details[i]).split('\n')[5].strip())
            user_review_cnt.append(str(info_details[i]).split('\n')[7].strip())
            user_name.append(re.findall('<p class="info__details__nickname" data(.*?)>(.*?)</p>', str(info_details[i]).split('\n')[0])[0][-1])
        time.sleep(3)
        
        star = bs.select(".stars")
        for i in range(len(star)-2):
            rating.append(str(star[i+2]).split('\n')[-2].strip())

        wd.close()

        prd_info = {
            "브랜드 link": brd_link,
            "브랜드명": brd_name,
            "브랜드 id": brd_id,
            "제품 link": prd_link,
            "제품 id": prd_id,
            "제품명": prd_name,
            "제품 용량": prd_volume,
            "제품 가격": prd_price,
            "유사 제품군 랭킹(int)": prd_grp_rank,
            "유사 제품군 랭킹(text)": prd_grp_text,
            "브랜드 내 랭킹(int)": prd_brd_rank,
            "제품 수상명": award_name,
            "제품 수상 부문": award_part,
            "제품 수상 순위": award_rank,
            "제품 설명": prd_desc,
            "제품 컬러/타입": color_type,
            "제품 판매처": store,
            "제품 성분구성: 위험도 미정 (갯수)": ingredient_unknown_cnt,
            "제품 성분구성: 위험도 미정 (성분(한글))": ingredient_unknown_kor_lst,
            "제품 성분구성: 위험도 미정 (성분(영어))": ingredient_unknown_eng_lst,
            "제품 성분구성: 위험도 미정 (성분(설명))": ingredient_unknown_desc_lst,
            "제품 성분구성: 낮은 위험도 (갯수)": ingredient_low_cnt,
            "제품 성분구성: 낮은 위험도 (성분(한글))": ingredient_low_kor_lst,
            "제품 성분구성: 낮은 위험도 (성분(영어))": ingredient_low_eng_lst,
            "제품 성분구성: 낮은 위험도 (성분(설명))": ingredient_low_desc_lst,
            "제품 성분구성: 중간 위험도 (갯수)": ingredient_mid_cnt,
            "제품 성분구성: 중간 위험도 (성분(한글))": ingredient_mid_kor_lst,
            "제품 성분구성: 중간 위험도(성분(영어))": ingredient_mid_eng_lst,
            "제품 성분구성: 중간 위험도 (성분(설명))": ingredient_mid_desc_lst,
            "제품 성분구성: 높은 위험도 (갯수)": ingredient_high_cnt,
            "제품 성분구성: 높은 위험도 (성분(한글))": ingredient_high_kor_lst,
            "제품 성분구성: 높은 위험도 (성분(영어))": ingredient_high_eng_lst,
            "제품 성분구성: 높은 위험도 (성분(설명))": ingredient_high_desc_lst,
            "제품 카테고리": category,
            "리뷰": review,
            "별점": rating,
            "리뷰 일자": review_date,
            "사용자 리뷰 개수": user_review_cnt,
            "사용자 피부 타입": user_skin_type,
            "사용자 나이": user_age,
            "사용자 성별": user_sex,
            "사용자 이름": user_name,
            "이미지 src": img_src
        }
        
        return prd_info
        
    except Exception as e:
        exception.append(prd_link)
        print(e)

In [13]:
prd_info = scrap_product_details("https://www.glowpick.com/products/124256")



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606


Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1650.16 Safari/537.36


Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [14]:
prd_info

{'브랜드 link': nan,
 '브랜드명': '솔더랩',
 '브랜드 id': nan,
 '제품 link': 'https://www.glowpick.com/products/124256',
 '제품 id': '124256',
 '제품명': '[단종] 클렌징 워터',
 '제품 용량': '250ml',
 '제품 가격': '18,000원',
 '유사 제품군 랭킹(int)': '',
 '유사 제품군 랭킹(text)': '',
 '브랜드 내 랭킹(int)': '',
 '제품 수상명': [],
 '제품 수상 부문': [],
 '제품 수상 순위': [],
 '제품 설명': '동해 해양심층수로 묵은 각질, 노폐물은 말끔하게 씻어내고 귀한 제주홍해삼이 탱글하게 차올라 촉촉함만 남기는 내 피부에 꼭 맞는 클렌징 워터',
 '제품 컬러/타입': '',
 '제품 판매처': [],
 '제품 성분구성: 위험도 미정 (갯수)': '0개',
 '제품 성분구성: 위험도 미정 (성분(한글))': [],
 '제품 성분구성: 위험도 미정 (성분(영어))': [],
 '제품 성분구성: 위험도 미정 (성분(설명))': [],
 '제품 성분구성: 낮은 위험도 (갯수)': '0개',
 '제품 성분구성: 낮은 위험도 (성분(한글))': [],
 '제품 성분구성: 낮은 위험도 (성분(영어))': [],
 '제품 성분구성: 낮은 위험도 (성분(설명))': [],
 '제품 성분구성: 중간 위험도 (갯수)': '0개',
 '제품 성분구성: 중간 위험도 (성분(한글))': [],
 '제품 성분구성: 중간 위험도(성분(영어))': [],
 '제품 성분구성: 중간 위험도 (성분(설명))': [],
 '제품 성분구성: 높은 위험도 (갯수)': '0개',
 '제품 성분구성: 높은 위험도 (성분(한글))': [],
 '제품 성분구성: 높은 위험도 (성분(영어))': [],
 '제품 성분구성: 높은 위험도 (성분(설명))': [],
 '제품 카테고리': ['클렌징', '메이크업클렌저', '클렌징워터'],
 '리뷰': [],

# V4.3에서 제품명이 업데이트된 제품 객체 수집

In [7]:
import pickle

with open("v4_update.txt", "rb") as f:
    v4_update_lst = pickle.load(f)

In [ ]:
v4_update_lst_0_800 = list(v4_update_lst.items())[:800]

In [ ]:
v4_update_lst_800_1661 = list(v4_update_lst.items())[800:]

In [ ]:
all_prd_info = pd.DataFrame(columns = ['브랜드 link','브랜드명','브랜드 id','제품 link','제품 id','제품명','제품 용량','제품 가격','유사 제품군 랭킹(int)',
                                         '유사 제품군 랭킹(text)','브랜드 내 랭킹(int)','제품 수상명','제품 수상 부문','제품 수상 순위','제품 설명','제품 컬러/타입',
                                         '제품 판매처','제품 성분구성: 위험도 미정 (갯수)','제품 성분구성: 위험도 미정 (성분(한글))',
                                         '제품 성분구성: 위험도 미정 (성분(영어))','제품 성분구성: 위험도 미정 (성분(설명))',
                                         '제품 성분구성: 낮은 위험도 (갯수)','제품 성분구성: 낮은 위험도 (성분(한글))',
                                         '제품 성분구성: 낮은 위험도 (성분(영어))','제품 성분구성: 낮은 위험도 (성분(설명))',
                                         '제품 성분구성: 중간 위험도 (갯수)','제품 성분구성: 중간 위험도 (성분(한글))',
                                         '제품 성분구성: 중간 위험도(성분(영어))','제품 성분구성: 중간 위험도 (성분(설명))',
                                         '제품 성분구성: 높은 위험도 (갯수)','제품 성분구성: 높은 위험도 (성분(한글))',
                                         '제품 성분구성: 높은 위험도 (성분(영어))','제품 성분구성: 높은 위험도 (성분(설명))',
                                         '제품 카테고리','리뷰','별점','리뷰 일자',
                                         '사용자 리뷰 개수','사용자 피부 타입','사용자 나이','사용자 성별','사용자 이름'])

In [ ]:
exception = []
i = 0

for _ in trange(len(v4_update_lst_0_800)):
    scrap_product_details(v4_update_lst_0_800[_][1])
    df_prd_info.loc[i] = prd_info
    i += 1